In [27]:
import pandas as pd
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from sklearn.preprocessing import LabelEncoder

In [9]:
_leukemia_dataset_file = '../Datasets/Leukemia_GSE9476.csv'
_rna_dataset_file = '../Datasets/METABRIC_RNA_Mutation.csv'

In [4]:
spark = SparkSession.builder.master(master = 'local[*]').appName("MLlib lab").getOrCreate()

In [14]:
df = pd.read_csv(_leukemia_dataset_file)
df.head(5)

,samples,type,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,...,AFFX-r2-Hs28SrRNA-5_at,AFFX-r2-Hs28SrRNA-M_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,1,Bone_Marrow_CD34,7.745245,7.811210,6.477916,8.841506,4.546941,7.957714,5.344999,4.673364,...,5.058849,6.810004,12.800060,12.718612,5.391512,4.666166,3.974759,3.656693,4.160622,4.139249
1,12,Bone_Marrow_CD34,8.087252,7.240673,8.584648,8.983571,4.548934,8.011652,5.579647,4.828184,...,4.436153,6.751471,12.472706,12.333593,5.379738,4.656786,4.188348,3.792535,4.204414,4.122700
2,13,Bone_Marrow_CD34,7.792056,7.549368,11.053504,8.909703,4.549328,8.237099,5.406489,4.615572,...,4.392061,6.086295,12.637384,12.499038,5.316604,4.600566,3.845561,3.635715,4.174199,4.067152
3,14,Bone_Marrow_CD34,7.767265,7.094460,11.816433,8.994654,4.697018,8.283412,5.582195,4.903684,...,4.633334,6.375991,12.903630,12.871454,5.179951,4.641952,3.991634,3.704587,4.149938,3.910150
4,15,Bone_Marrow_CD34,8.010117,7.405281,6.656049,9.050682,4.514986,8.377046,5.493713,4.860754,...,5.305192,6.700453,12.949352,12.782515,5.341689,4.560315,3.887020,3.629853,4.127513,4.004316


In [15]:
LE = LabelEncoder()
df['type'] = LE.fit_transform(df['type'])

In [16]:
sparkDF = spark.createDataFrame(df)

In [17]:
# sparkDF = spark.read.option("maxColumns", 22285).csv(_leukemia_dataset_file)

In [23]:
X = sparkDF.drop('type')
y = sparkDF.select(sparkDF['type'])

In [24]:
X.printSchema()

root
 |-- samples: long (nullable = true)
 |-- 1007_s_at: double (nullable = true)
 |-- 1053_at: double (nullable = true)
 |-- 117_at: double (nullable = true)
 |-- 121_at: double (nullable = true)
 |-- 1255_g_at: double (nullable = true)
 |-- 1294_at: double (nullable = true)
 |-- 1316_at: double (nullable = true)
 |-- 1320_at: double (nullable = true)
 |-- 1405_i_at: double (nullable = true)
 |-- 1431_at: double (nullable = true)
 |-- 1438_at: double (nullable = true)
 |-- 1487_at: double (nullable = true)
 |-- 1494_f_at: double (nullable = true)
 |-- 1598_g_at: double (nullable = true)
 |-- 160020_at: double (nullable = true)
 |-- 1729_at: double (nullable = true)
 |-- 1773_at: double (nullable = true)
 |-- 177_at: double (nullable = true)
 |-- 179_at: double (nullable = true)
 |-- 1861_at: double (nullable = true)
 |-- 200000_s_at: double (nullable = true)
 |-- 200001_at: double (nullable = true)
 |-- 200002_at: double (nullable = true)
 |-- 200003_s_at: double (nullable = true)
 |

In [25]:
y.printSchema()

root
 |-- type: long (nullable = true)



In [28]:
vecAssembler = VectorAssembler(inputCols=X.columns, outputCol="features")

In [29]:
stream_df = vecAssembler.transform(X)

In [30]:
stream_df.show()

Py4JJavaError: An error occurred while calling o112.showString.
: java.lang.OutOfMemoryError: Java heap space
	at java.lang.AbstractStringBuilder.<init>(AbstractStringBuilder.java:68)
	at java.lang.StringBuilder.<init>(StringBuilder.java:107)
	at org.apache.spark.sql.catalyst.expressions.objects.AssertNotNull.<init>(objects.scala:1789)
	at org.apache.spark.sql.catalyst.encoders.ExpressionEncoder$$anonfun$4.applyOrElse(ExpressionEncoder.scala:250)
	at org.apache.spark.sql.catalyst.encoders.ExpressionEncoder$$anonfun$4.applyOrElse(ExpressionEncoder.scala:246)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformUpWithPruning$2(TreeNode.scala:535)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$Lambda$1155/3371873.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:82)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUpWithPruning(TreeNode.scala:535)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anon